In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
filename = '/home/dparks/Downloads/rmotc/DataSets/Seismic/CD files/3D_Seismic/filt_mig.sgy'

In [4]:
from obspy.io.segy.segy import _read_segy
stream = _read_segy(filename, headonly=True)

In [5]:

traces_with_indexes = []

for trace in stream.traces[:10000]:
    x = trace.header.x_coordinate_of_ensemble_position_of_this_trace - 1
    y = trace.header.y_coordinate_of_ensemble_position_of_this_trace - 1

    traces_with_indexes.append((x,y, trace.data))


In [6]:
x_max = max(traces_with_indexes, key = lambda x: x[0])[0]
y_max = max(traces_with_indexes, key = lambda x: x[1])[1]

print(x_max, y_max)

53 187


In [7]:
x_len = x_max + 1
y_len = y_max + 1
s_len = len(traces_with_indexes[0][2])
threeDSamples = np.zeros((x_len, y_len, s_len))

In [8]:
for xy_trace in traces_with_indexes:
    x = xy_trace[0]
    y = xy_trace[1]
    samples = xy_trace[2]
    threeDSamples[x, y] = samples

In [9]:
vm = np.percentile(threeDSamples, 99)

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy.signal import hilbert
from scipy import interpolate
from scipy import signal

samples   = stream.binary_file_header.number_of_samples_per_data_trace
samples = samples -1

def ampspec(signal,sr,smooth=False):
    '''
    ampspec (C) aadm 2016
    Calculates amplitude spectrum of a signal with FFT optionally smoothed via cubic interpolation.

    INPUT
    signal: 1D numpy array
    sr: sample rate in ms
    smooth: True or False

    OUTPUT
    freq: frequency
    amp: amplitude
    '''

    SIGNAL = np.fft.fft(signal)
    freq = np.fft.fftfreq(signal.size, d=sr*0.001)
    # keep = (freq>=0) & (freq<=freq.max()/2)
    keep = freq>=0
    SIGNAL = np.abs(SIGNAL[keep])
    freq = freq[keep]
    if smooth:
        # freq0=np.linspace(freq.min(),freq.max()/2,freq.size*10)
        freq0=np.linspace(freq.min(),freq.max(),freq.size*10)
        f = interpolate.interp1d(freq, SIGNAL, kind='cubic')
        return freq0, f(freq0)
    else:
        return freq, SIGNAL


def fullspec(data,sr):
    '''
    fullspec (C) aadm 2016
    Calculates amplitude spectrum of 2D numpy array.

    INPUT
    data: 2D numpy array, shape=(samples, traces)
    sr: sample rate in ms

    OUTPUT
    freq: frequency
    amp: amplitude
    db: amplitude in dB scale
    f_peak: average peak frequency
    '''
    amps, peaks = [], []
    for i in range(data.shape[1]):
        trace = data[:,i]
        freq, amp = ampspec(trace,sr)
        peak = freq[np.argmax(amp)]
        amps.append(amp)
        peaks.append(peak)
    amp0 = np.mean(np.dstack(amps), axis=-1)
    amp0 = np.squeeze(amp0)
    db0 = 20 * np.log10(amp0)
    db0 = db0 - np.amax(db0)
    f_peak = np.mean(peaks)
    print('freq peak: {:.2f} Hz'.format(f_peak))
    return freq,amp0,db0,f_peak

def plot_ampspec(freq,amp,f_peak,name=None):
    '''
    plot_ampspec (C) aadm 2016
    Plots amplitude spectrum calculated with fullspec (aageofisica.py).

    INPUT
    freq: frequency
    amp: amplitude
    f_peak: average peak frequency
    '''
    db = 20 * np.log10(amp)
    db = db - np.amax(db)
    f, ax = plt.subplots(nrows=1,ncols=2,figsize=(12,5),facecolor='w')
    ax[0].plot(freq, amp, '-k', lw=2)
    ax[0].set_ylabel('Power')
    ax[1].plot(freq, db, '-k', lw=2)
    ax[1].set_ylabel('Power (dB)')
    for aa in ax:
        aa.set_xlabel('Frequency (Hz)')
        aa.set_xlim([0,np.amax(freq)/1.5])
        aa.grid()
        aa.axvline(f_peak, color='r', ls='-')
        if name!=None:
            aa.set_title(name, fontsize=16)

def plot_ampspec2(freq,amp1,amp2,f_peak1,f_peak2,name1=None,name2=None):
    '''
    plot_ampspec2 (C) aadm 2016
    Plots overlay of 2 amplitude spectra calculated with fullspec (aageofisica.py).

    INPUT
    freq: frequency
    amp1, amp2: amplitude spectra
    f_peak1, f_peak2: average peak frequency
    '''
    db1 = 20 * np.log10(amp1)
    db1 = db1 - np.amax(db1)
    db2 = 20 * np.log10(amp2)
    db2 = db2 - np.amax(db2)
    f, ax = plt.subplots(nrows=1,ncols=2,figsize=(12,5),facecolor='w')
    if name1!=None:
        label1='{:s} Fp={:.0f} Hz'.format(name1,f_peak1)
        label2='{:s} Fp={:.0f} Hz'.format(name2,f_peak2)
    else:
        label1='Fp={:.0f} Hz'.format(f_peak1)
        label2='Fp={:.0f} Hz'.format(f_peak2)
    ax[0].plot(freq, amp1, '-k', lw=2, label=label1)
    ax[0].plot(freq, amp2, '-r', lw=2, label=label2)
    ax[0].fill_between(freq,0,amp1,lw=0, facecolor='k',alpha=0.25)
    ax[0].fill_between(freq,0,amp2,lw=0, facecolor='r',alpha=0.25)
    ax[0].set_ylabel('Power')
    ax[1].plot(freq, db1, '-k', lw=2, label=label1)
    ax[1].plot(freq, db2, '-r', lw=2,label=label2)
    lower_limit=np.min(ax[1].get_ylim())
    ax[1].fill_between(freq, db1, lower_limit, lw=0, facecolor='k', alpha=0.25)
    ax[1].fill_between(freq, db2, lower_limit, lw=0, facecolor='r', alpha=0.25)
    ax[1].set_ylabel('Power (dB)')
    for aa in ax:
        aa.set_xlabel('Frequency (Hz)')
        aa.set_xlim([0,np.amax(freq)/1.5])
        aa.grid()
        aa.axvline(f_peak1, color='k', ls='-')
        aa.axvline(f_peak2, color='r', ls='-')
        aa.legend(fontsize='small')




def inst_attribute(data):
    rows, columns = data.shape
    new_data = np.zeros((rows,columns))
    for i in range(columns):
        analytic_signal = hilbert(data[:, i])
        amplitude_envelope = np.abs(analytic_signal)
        new_data[:,i] = amplitude_envelope
    return new_data




def f(x):
    plt.figure(figsize=(16,8))
    data = threeDSamples.T
    chop = data[:,:,x]
    new_data = inst_attribute(chop)
    plt.imshow(new_data, interpolation='bilinear', cmap="seismic", vmin=-vm, vmax=vm, aspect='auto')
    plt.colorbar(shrink=0.5)
    plt.show()
    #return x
#interact(myplot, start = 0, x_max=10);
interact(f, x=widgets.IntSlider(min=0,max=x_max,step=1,value=10));


def ff(x):
    plt.figure(figsize=(16,8))
    data = threeDSamples.T
    plt.imshow(data[:,x,:], interpolation='bilinear', cmap="seismic", vmin=-vm, vmax=vm, aspect='auto')
    plt.colorbar(shrink=0.5)
    plt.show()
    #return x
#interact(myplot, start = 0, x_max=10);
interact(ff, x=widgets.IntSlider(min=0,max=y_max,step=1,value=10));

def fff(x):
    plt.figure(figsize=(16,8))
    data = threeDSamples.T
    plt.imshow(data[x,:,:], interpolation='bilinear', cmap="gray", vmin=-vm, vmax=vm, aspect='auto')
    plt.colorbar(shrink=0.5)
    plt.show()
    #return x
#interact(myplot, start = 0, x_max=10);
interact(fff, x=widgets.IntSlider(min=0,max=samples,step=1,value=10));

def spec_decomp(x):
    
    data = threeDSamples.T
    chop = data[x,:,:]
    
    sampling_rate = stream.binary_file_header.sample_interval_in_microseconds/1000 # converts sample rate to milliseconds :-)
    # filter definition
    sample_frequency = 1/(sampling_rate*.001) # the 0.001 convets sample rate from ms to s
    nyq = 0.5 * sample_frequency
    print ('The sample frequency is: {} Hz'.format(sample_frequency))
    print ('The nyquist frequency is: {} Hz'.format(nyq))
    order = 7
    cuttoff = 20 #my cutoff frequency
    B,A = signal.butter(order, cuttoff/nyq, btype='lowpass', analog=False) #btype='lowpass', #we wont use this coz no min max limit of freq..
    lowcut = 40
    highcut= 100

    fmin    = lowcut/nyq
    fmax    = highcut/nyq

    #Make filter shape
    corners=4
    B, A = signal.iirfilter(corners, [fmin,fmax], btype='band', ftype='butter')

    # filter application
    line1_filt = signal.filtfilt(B,A,chop,axis=0)
    
    freq,amp,db,fpeak = fullspec(line1_filt,sampling_rate)
    plot_ampspec(freq,amp,fpeak,name=None)
    
    plt.figure(figsize=(16,8))
    plt.imshow(line1_filt, interpolation='bilinear', cmap="seismic", vmin=-vm, vmax=vm, aspect='auto')
    plt.colorbar(shrink=0.5)
    plt.show()
    #return x
#interact(myplot, start = 0, x_max=10);
interact(spec_decomp, x=widgets.IntSlider(min=40,max=100,step=1,value=10));#min max must match lowcut highcut freq in filter



interactive(children=(IntSlider(value=10, description='x', max=53), Output()), _dom_classes=('widget-interact'…

interactive(children=(IntSlider(value=10, description='x', max=187), Output()), _dom_classes=('widget-interact…

interactive(children=(IntSlider(value=10, description='x', max=1500), Output()), _dom_classes=('widget-interac…

interactive(children=(IntSlider(value=40, description='x', min=40), Output()), _dom_classes=('widget-interact'…

In [9]:
arr = np.arange(100)

In [10]:
arr = arr.reshape(2,5,10)
print(arr)

[[[ 0  1  2  3  4  5  6  7  8  9]
  [10 11 12 13 14 15 16 17 18 19]
  [20 21 22 23 24 25 26 27 28 29]
  [30 31 32 33 34 35 36 37 38 39]
  [40 41 42 43 44 45 46 47 48 49]]

 [[50 51 52 53 54 55 56 57 58 59]
  [60 61 62 63 64 65 66 67 68 69]
  [70 71 72 73 74 75 76 77 78 79]
  [80 81 82 83 84 85 86 87 88 89]
  [90 91 92 93 94 95 96 97 98 99]]]


In [11]:
print(arr[0])

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]
 [20 21 22 23 24 25 26 27 28 29]
 [30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]]


In [12]:
print(arr.T.shape)
print(arr.T)

(10, 5, 2)
[[[ 0 50]
  [10 60]
  [20 70]
  [30 80]
  [40 90]]

 [[ 1 51]
  [11 61]
  [21 71]
  [31 81]
  [41 91]]

 [[ 2 52]
  [12 62]
  [22 72]
  [32 82]
  [42 92]]

 [[ 3 53]
  [13 63]
  [23 73]
  [33 83]
  [43 93]]

 [[ 4 54]
  [14 64]
  [24 74]
  [34 84]
  [44 94]]

 [[ 5 55]
  [15 65]
  [25 75]
  [35 85]
  [45 95]]

 [[ 6 56]
  [16 66]
  [26 76]
  [36 86]
  [46 96]]

 [[ 7 57]
  [17 67]
  [27 77]
  [37 87]
  [47 97]]

 [[ 8 58]
  [18 68]
  [28 78]
  [38 88]
  [48 98]]

 [[ 9 59]
  [19 69]
  [29 79]
  [39 89]
  [49 99]]]
